<a href="https://colab.research.google.com/github/saakolch/procedure_of_extracting_data/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**We have to predict the patient condition based on the review. It would be multi-label classification with reviews as features with tokens and conditions as labels with numbers in our training**

In [1]:
!pip install datasets evaluate transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 31.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
!wget "https://datasets-server.huggingface.co/splits?dataset=forwins%2FDrug-Review-Dataset"

--2024-05-19 05:49:45--  https://datasets-server.huggingface.co/splits?dataset=forwins%2FDrug-Review-Dataset
Resolving datasets-server.huggingface.co (datasets-server.huggingface.co)... 13.35.7.51, 13.35.7.22, 13.35.7.60, ...
Connecting to datasets-server.huggingface.co (datasets-server.huggingface.co)|13.35.7.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 272 [application/json]
Saving to: ‘splits?dataset=forwins%2FDrug-Review-Dataset’

splits?dataset=forw 100%[===================>]     272  --.-KB/s    in 0s      

2024-05-19 05:49:46 (12.1 MB/s) - ‘splits?dataset=forwins%2FDrug-Review-Dataset’ saved [272/272]



In [3]:
from datasets import load_dataset
from transformers import DataCollatorWithPadding, AutoTokenizer

drug_dataset = load_dataset("forwins/Drug-Review-Dataset")
drug_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/110811 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/27703 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/46108 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

In [4]:
drug_dataset['train']['condition'][:3]

['keratoconjunctivitis sicca', 'birth control', 'birth control']

In [25]:
import numpy as np

In [37]:
import numpy as np
unique = set(drug_dataset['train']["condition"])
size_matrix = np.zeros((len(drug_dataset['train']['review']), len(unique)))
size_matrix.shape

(110811, 791)

We need convert our data to the tokens

In [5]:
checkpoints = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

def preprocess_func(data):
    tokenizer(data['review'], truncation=True)
    unique = set(drug_dataset['train']["condition"])
    size_matrix = np.zeros((len(drug_dataset['train']['review']), len(unique)))

tokenized_data = drug_dataset.map(preprocess_func, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/110811 [00:00<?, ? examples/s]

Map:   0%|          | 0/27703 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

We are going to delete all of the columns except condition one, it would be used as our labels

In [6]:
encoded_data = encoded_dataset.remove_columns(['patient_id', 'drugName', 'review', 'rating', 'date', 'usefulCount', 'review_length','condition'])

encoded_data.set_format('torch')

encoded_data['train'].column_names

NameError: name 'encoded_dataset' is not defined

In [ ]:
encoded_data.set_format('torch')

encoded_data['train'].column_names

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    encoded_data['train'],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator
)
eval_dataloader = DataLoader(
    encoded_data['validation'],
    batch_size=8,
    collate_fn=data_collator
)

To quickly examinate the data for errors:

In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k,v in batch.items()}

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-dbmz-uncased", problem_type="multi_label_classification", num_labels=

In [ ]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)